In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("train.csv", index_col=[0])

In [4]:
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
df["Weight"] = np.random.rand(len(df), 1)

df["bool"] = np.random.choice([False, True], size=len(df))

In [6]:
def weighted_avg(x, wts):
    return np.average(x, weights=wts, axis=0)

def weighted_std(x, wts):
    average = np.average(x, weights=wts, axis=0)
    xiance = np.average((x - average) ** 2, weights=wts, axis=0)
    return np.sqrt(xiance)

def weighted_skew(x, wts):
    return (np.average((x - weighted_avg(x, wts))**3, weights=wts, axis=0) /
            weighted_avg(x, wts)**(1.5))

def weighted_kurtosis(x, wts):
    return (np.average((x - weighted_avg(x, wts))**4, weights=wts, axis=0) /
            weighted_avg(x, wts)**(2))

In [7]:
funcs = [weighted_avg, weighted_std, weighted_skew, weighted_kurtosis]
metrics = ["LotArea", "LotFrontage", "SalePrice"]

In [8]:
df.dropna(subset=metrics, inplace=True)

In [38]:
def weighted_functions(df, funcs, metrics):
    names = [func.__name__ for func in funcs]
    groups = []
    
    for func in funcs:
        gb = df.groupby(["MSZoning", "bool"]).apply(lambda x: pd.Series(func(x[metrics], x["Weight"]), metrics)).T
        groups.append(gb)
        
    gb = df.groupby(["MSZoning", "SaleCondition"]).apply(lambda x: pd.Series(np.median(x[metrics]), metrics)).T
    groups.append(gb)
    names.append("median")
    
    return pd.concat(groups, axis=1, keys=names).T

In [39]:
a = df.loc[(df["bool"] == True) & (df["MSZoning"] == "RM"), ["SalePrice", "Weight"]]

In [40]:
np.average(a["SalePrice"], weights=a["Weight"])

127407.16945257325

In [41]:
res = weighted_functions(df, funcs, metrics=metrics)
res.loc["weighted_avg"]

LotArea  LotFrontage      SalePrice
MSZoning                                                
C (all)  False   9521.766561    62.264570   54564.859322
         True    9108.867062    74.098858   74966.732416
FV       False   7226.602160    62.776769  218887.636493
         True    7159.666700    60.238650  209648.591293
RH       False   6437.195846    63.475097  107503.595070
         True    6549.242764    59.276199  144897.949720
RL       False  10743.306030    75.463948  191371.722286
         True   10978.739944    74.289485  194371.979651
RM       False   5835.481848    49.460106  117098.445735
         True    6836.654519    53.020287  127407.169453